In [231]:
import os,sys
from pickle import HIGHEST_PROTOCOL

# Path stuff
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\PSSPY37")
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\PSSBIN")
os.environ['PATH'] = (r"C:\Program Files\PTI\PSSE35\35.3\PSSPY37;" 
  + r"C:\Program Files\PTI\PSSE35\35.3\PSSBIN;" 
  + r"C:\Program Files\PTI\PSSE35\35.3\EXAMPLE;" + os.environ['PATH'])


import pandas as pd
import psse35
psse35.set_minor(3)
import numpy as np
import matplotlib.pyplot as plt
import psspy
psspy.psseinit(100)
import dyntools # The command for Dynamic Tools

In [232]:
# Read in case file 
case = r"../input_files/case.sav"
psspy.case(case)





 The Saved Case in file C:\Users\alexb\research\WEC-GRID\input_files\case.sav was saved on THU, JUL 16 2020  14:57


0

In [233]:
psspy.fnsl()


  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0         0.0000(      1     )    0.0000(      1     )
                                                             0.00000(      1     )   0.00000(      1     )
   1         0.0000(      1     )    0.0000(      1     )
                                                             0.00000(      1     )   0.00000(      1     )
   2         0.0000(      1     )    0.0000(      1     )


 Reached tolerance in 2 iterations

 Largest mismatch:      0.00 MW      0.00 Mvar      0.00 MVA at bus 3 [MV_2        25.000]
 System total absolute mismatch:                    0.00 MVA

 SWING BUS SUMMARY:
   BUS#-SCT X-- NAME --X BASKV      PGEN     PMAX    PMIN      QGEN     QMAX    QMIN
      3     MV_2        25.000      -5.0   9999.0 -9999.0      -1.6   9999.0 -9999.0


0

In [234]:
dynamic_case_file = "../input_files/dynamics.dyr"

In [247]:
psspy.cong()
# Solve for dynamics
psspy.ordr()
psspy.fact()
psspy.tysl()
# Add Dynamics data file
psspy.dyre_new(dyrefile=dynamic_case_file)
# Add channels and parameters
# BUS VOLTAGE
psspy.chsb(sid=0, all=1, status=[-1, -1, -1, 1, 13, 0])
# Active and Reactive Power Flow
psspy.chsb(sid=0, all=1, status=[-1, -1, -1, 1, 16, 0])


# Save snapshoot
psspy.snap(sfile='C:\\Users\\alexb\\research\\WEC-GRID\\output_files\\test.out')

# Initialize
psspy.strt(outfile='C:\\Users\\alexb\\research\\WEC-GRID\\output_files\\test.out')
psspy.run(tpause=1)
# Set simulation parameter step size
psspy.dynamics_solution_params(realar3=0.01)
# 3-Phase Fault ( In our example is the Swing with the identification number 3)
# psspy.dist_bus_fault(ibus = 3)
psspy.dist_bus_fault_2(ibus=3, units=1, values1=0, values2= -2e11) 

# Run to (...seconds) (Time that runs fault)
time = 0.15
psspy.run(tpause=time)

# Clear fault
psspy.dist_clear_fault()

# Run to ...seconds (time for simulation to run after the fault is cleared)
time = 2 
psspy.run(tpause=time)


 Messages for api CONG
 Generators are already "converted"   (001416)
 Diagonals = 2   Off-diagonals = 1   Maximum size = 2

 2 diagonal and 1 off-diagonal elements

 ITER DELTAV/TOL X----------- AT BUS -----------X  REAL(DELTAV)  IMAG(DELTAV)
    1     0.000                                     0.0000E+00    0.0000E+00

 Reached tolerance in 1 iterations

 Largest mismatch:      0.00 MW     -0.00 Mvar      0.00 MVA at bus 3 [MV_2        25.000]
 System total absolute mismatch:                    0.00 MVA
 Out of file data--switch to terminal input mode

 Generator models use:
  CONS         1-      2
  STATES       1-      2

 Renewable Generator models use:
  CONS         3-     11
  STATES       3-      5
  VARS         1-      3

 Renewable Electrical models use:
  CONS        12-     34
  STATES       6-     15
  VARS         4-      7
  ICONS        1-      4


 SUMMARY OF MODELS READ:

 GENS:  GENCLS
           1

 REGEN: WT4G1
           1

 REELC: WT4E1
           1

 NEXT AVA

0

In [248]:
outfile = 'C:\\Users\\alexb\\research\\WEC-GRID\\output_files\\test.out'

In [249]:
outlst = [outfile]

In [257]:
chnfobj = dyntools.CHNF(outlst)

In [258]:
sh_ttl, ch_id, ch_data = chnfobj.get_data()

In [259]:
ch_data

OrderedDict([('time',
              [-0.019999999552965164,
               -0.009999999776482582,
               0.0,
               0.009999999776482582,
               0.019999999552965164,
               0.029999999329447746,
               0.03999999910593033,
               0.04999999701976776,
               0.059999994933605194,
               0.06999999284744263,
               0.07999999076128006,
               0.08999998867511749,
               0.09999998658895493,
               0.10999998450279236,
               0.11999998241662979,
               0.12999998033046722,
               0.13999998569488525,
               0.14999999105930328,
               0.1599999964237213,
               0.17000000178813934,
               0.18000000715255737,
               0.1900000125169754,
               0.20000001788139343,
               0.21000002324581146,
               0.2200000286102295,
               0.23000003397464752,
               0.24000003933906555,
               0.

In [260]:
chnfobj.get_id()

('\n',
 OrderedDict([('time', 'Time(s)'),
              (1, 'VOLT 1 [WIND 0.6000]'),
              (2, 'VOLT 2 [MV 25.000]'),
              (3, 'VOLT 3 [MV_2 25.000]'),
              (4, "POWR 1 TO 2 CKT '1 '"),
              (5, "VARS 1 TO 2 CKT '1 '"),
              (6, "POWR 2 TO 1 CKT '1 '"),
              (7, "VARS 2 TO 1 CKT '1 '"),
              (8, "POWR 2 TO 3 CKT '1 '"),
              (9, "VARS 2 TO 3 CKT '1 '"),
              (10, "POWR 3 TO 2 CKT '1 '"),
              (11, "VARS 3 TO 2 CKT '1 '"),
              (12, 'VOLT 1 [WIND 0.6000]'),
              (13, 'VOLT 2 [MV 25.000]'),
              (14, 'VOLT 3 [MV_2 25.000]'),
              (15, "POWR 1 TO 2 CKT '1 '"),
              (16, "VARS 1 TO 2 CKT '1 '"),
              (17, "POWR 2 TO 1 CKT '1 '"),
              (18, "VARS 2 TO 1 CKT '1 '"),
              (19, "POWR 2 TO 3 CKT '1 '"),
              (20, "VARS 2 TO 3 CKT '1 '"),
              (21, "POWR 3 TO 2 CKT '1 '"),
              (22, "VARS 3 TO 2 CKT '1 '"),
   

In [261]:
chnfobj.get_range()

OrderedDict([('time',
              {'min': 0.0,
               'max': 0.9999993443489075,
               'v0': -0.019999999552965164}),
             (1,
              {'min': 1.0000015497207642,
               'max': 1.0000015497207642,
               'v0': 1.0000015497207642}),
             (2,
              {'min': 0.9999999403953552,
               'max': 0.9999999403953552,
               'v0': 0.9999999403953552}),
             (3,
              {'min': 0.9999999403953552,
               'max': 0.9999999403953552,
               'v0': 0.9999999403953552}),
             (4,
              {'min': 4.999999523162842,
               'max': 4.999999523162842,
               'v0': 4.999999523162842}),
             (5,
              {'min': 1.643420696258545,
               'max': 1.643420696258545,
               'v0': 1.643420696258545}),
             (6,
              {'min': -4.999999523162842,
               'max': -4.999999523162842,
               'v0': -4.999999523162842}),
     

In [262]:
chnfobj.get_scale()

OrderedDict([('time', {'min': 0, 'max': 1, 'v0': -0.019999999552965164}),
             (1, {'min': 0.5, 'max': 1.5, 'v0': 1.0000015497207642}),
             (2, {'min': 0.5, 'max': 1.5, 'v0': 0.9999999403953552}),
             (3, {'min': 0.5, 'max': 1.5, 'v0': 0.9999999403953552}),
             (4, {'min': 0.0, 'max': 10.0, 'v0': 4.999999523162842}),
             (5, {'min': 1.0, 'max': 2.0, 'v0': 1.643420696258545}),
             (6, {'min': -10.0, 'max': 0.0, 'v0': -4.999999523162842}),
             (7, {'min': -2.0, 'max': -1.0, 'v0': -1.6433931589126587}),
             (8, {'min': 0.0, 'max': 10.0, 'v0': 4.999999523162842}),
             (9, {'min': 1.0, 'max': 2.0, 'v0': 1.6433924436569214}),
             (10, {'min': -10.0, 'max': 0.0, 'v0': -4.999999523162842}),
             (11, {'min': -2.0, 'max': -1.0, 'v0': -1.6433924436569214}),
             (12, {'min': 0.5, 'max': 1.5, 'v0': 1.0000015497207642}),
             (13, {'min': 0.5, 'max': 1.5, 'v0': 0.9999999403953552}),
   

In [263]:
chnfobj.print_scale()


 File : c:\users\alexb\research\wec-grid\output_files\test.out

                                       ACTUAL               SCALED     
 CHN#    CHANNEL IDENTIFIER       MINIMUM   MAXIMUM    MINIMUM  MAXIMUM
         Time(s)                        0  0.999999          0        1
  1      VOLT 1 [WIND 0.6000]           1         1        0.5      1.5
  2      VOLT 2 [MV 25.000]             1         1        0.5      1.5
  3      VOLT 3 [MV_2 25.000]           1         1        0.5      1.5
  4      POWR 1 TO 2 CKT '1 '           5         5          0       10
  5      VARS 1 TO 2 CKT '1 '     1.64342   1.64342          1        2
  6      POWR 2 TO 1 CKT '1 '          -5        -5        -10        0
  7      VARS 2 TO 1 CKT '1 '    -1.64339  -1.64339         -2       -1
  8      POWR 2 TO 3 CKT '1 '           5         5          0       10
  9      VARS 2 TO 3 CKT '1 '     1.64339   1.64339          1        2
  10     POWR 3 TO 2 CKT '1 '          -5        -5        -10        0

In [264]:
chnfobj.txtout(channels=[1,3])

In [267]:
chnfobj.plots_show()